In [39]:
from blendsql.db import SQLite
from research.constants import EvalField
import json
from blendsql.models import AzureOpenaiLLM
from tqdm.notebook import tqdm
from blendsql import LLMQA, LLMMap, LLMJoin, blend

In [40]:
from blendsql._program import Program
from guidance import gen

BASE_SYSTEM_PROMPT = """
This is a hybrid question answering task. The goal of this task is to answer the question given a table (`w`) and corresponding passages (`docs`).
Be as succinct as possible in answering the given question, do not include explanation.
"""

class EndtoEndProgram(Program):
    def __call__(self, serialized_db: str, question: str, **kwargs):
        with self.systemcontext:
            self.model += BASE_SYSTEM_PROMPT
        with self.usercontext:
            self.model += f"Context:\n{serialized_db}\n\n"
            self.model += f"Question: {question}\n"
            self.model += f"Answer:\n"
        return self.model._current_prompt()

BASE_SYSTEM_PROMPT = """
Generate BlendSQL given the question, table, and passages to answer the question correctly.
BlendSQL is a superset of SQLite, which adds external function calls for information not found within native SQLite.
These external functions should be wrapped in double curly brackets.

{ingredients_prompt}

Additionally, we have the table `documents` at our disposal, which contains Wikipedia articles providing more details about the values in our table.
ONLY use BlendSQL ingredients if necessary.
Answer parts of the question in vanilla SQL, if possible.
"""
with open("../research/prompts/hybridqa/few_shot.txt", "r") as f:
    few_shot_prompt = f.read()

with open("../research/prompts/hybridqa/ingredients.txt", "r") as f:
    ingredients_prompt = f.read()

class ParserProgram(Program):
    def __call__(self, serialized_db: str, question: str, bridge_hints: str = None, **kwargs):
        with self.systemcontext:
            self.model += BASE_SYSTEM_PROMPT.format(ingredients_prompt=ingredients_prompt)
        with self.usercontext:
            self.model += f"{few_shot_prompt}\n\n"
            self.model += f"{serialized_db}\n\n"
            if bridge_hints:
                self.model += f"Here are some values that may be useful: {bridge_hints}\n"
            self.model += f"Q: {question}\n"
            self.model += f"BlendSQL:\n"
        return self.model._current_prompt()

In [42]:
with open("../research/outputs/paper-results/hybridqa/gpt-4-blendsql-plus-pp/predictions.json", "r") as f:
    predictions = json.load(f)

In [35]:
num_tokens = 0
model = AzureOpenaiLLM("gpt-4", env="..")
for item in tqdm(predictions):
    db = SQLite( "../" + item[EvalField.DB_PATH])
    question = item[EvalField.QUESTION]
    prompt = EndtoEndProgram(model=model.model, serialized_db=db.to_serialized(whole_table=True), question=question)
    num_tokens += len(model.tokenizer.encode(prompt))
print("Average: end-to-end, no truncation")
print(num_tokens / len(predictions))

  0%|          | 0/3466 [00:00<?, ?it/s]

Average: end-to-end, no truncation
7857.350548182343


In [36]:
num_tokens = 0
model = AzureOpenaiLLM("gpt-4", env="..")
for item in tqdm(predictions):
    db = SQLite( "../" + item[EvalField.DB_PATH])
    question = item[EvalField.QUESTION]
    prompt = EndtoEndProgram(model=model.model, serialized_db=db.to_serialized(whole_table=True, truncate_content=400), question=question)
    num_tokens += len(model.tokenizer.encode(prompt))
print("Average: end-to-end, truncated to 400:")
print(num_tokens / len(predictions))

  0%|          | 0/3466 [00:00<?, ?it/s]

Average: end-to-end, truncated to 400:
3910.5545297172534


In [54]:
num_tokens = 0
model = AzureOpenaiLLM("gpt-4", env="..")
for item in tqdm(predictions):
    db = SQLite( "../" + item[EvalField.DB_PATH])
    question = item[EvalField.QUESTION]
    prompt = ParserProgram(model=model.model, serialized_db=db.to_serialized(num_rows=3), question=question)
    num_tokens += len(model.tokenizer.encode(prompt))
    model = AzureOpenaiLLM("gpt-4", env="..", caching=False)
    try:
        res = blend(
            query=item['pred_sql'],
            db=db,
            ingredients={LLMMap, LLMQA, LLMJoin},
            blender=model,
            infer_gen_constraints=True,
            silence_db_exec_errors=False,
            verbose=False,
        )
        num_tokens += res.meta.num_prompt_tokens
    except:
        pass
print("Average: BlendSQL, 3 rows:")
print(num_tokens / len(predictions))

  0%|          | 0/3466 [00:00<?, ?it/s]

Something went wrong!
Execution failed on sql 'SELECT * FROM "documents" WHERE w."capacity" = (SELECT MAX(w."capacity") AS _col_0 FROM w)': no such column: w.capacity
Something went wrong!
Execution failed on sql 'SELECT * FROM "documents" WHERE w."year" = (SELECT MAX(w."year") AS _col_0 FROM w)': no such column: w.year
Something went wrong!
Execution failed on sql 'SELECT * FROM "documents" WHERE w."year" BETWEEN 1983 AND 1984': no such column: w.year
Something went wrong!
Execution failed on sql 'SELECT * FROM "documents" WHERE w."losses" = (SELECT MIN(w."losses") AS _col_0 FROM w)': no such column: w.losses
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...
Refreshing the access tokens...Refreshing the access tokens...


KeyboardInterrupt



In [51]:
num_tokens

4673

In [38]:
print(db.to_serialized(num_rows=3))

CREATE TABLE "w" (
"index" INTEGER,
  "program" TEXT,
  "network" TEXT,
  "debut date" TEXT
)
/*
3 example rows:
SELECT * FROM "w" LIMIT 3
 index               program network debut date
     0             freefonix    abc3  2 january
     1       / the octonauts    abc2  2 january
     2 facejacker facejacker   sbs 1 10 january
*/

	CREATE VIRTUAL TABLE documents USING fts5(title, content, tokenize = 'trigram')


In [44]:
predictions[0]

{'id': '00153f694413a536',
 'dataset_vars': {'table_id': 'List_of_National_Football_League_rushing_yards_leaders_0',
  'gold_passage': "passages: Emmitt Smith: Smith led the league in rushing and won the Super Bowl in the same year three times ( 1992 , 1993 , and 1995 ) when to that point it had never been done . | Walter Payton: Walter Jerry Payton ( July 25 , 1954 - November 1 , 1999 ) was an American professional football player who was a running back for the Chicago Bears of the National Football League ( NFL ) for thirteen seasons . | San Francisco 49ers: The team began play at Kezar Stadium in San Francisco before moving across town to Candlestick Park in 1970 and then to Levi 's Stadium in Santa Clara in 2014 . The 49ers won five Super Bowl championships between 1981 and 1994 , led by Hall of Famers Joe Montana , Jerry Rice , Ronnie Lott , Steve Young , and coaches Bill Walsh and George Seifert . | Denver Broncos: They have won eight AFC Championships ( 1977 , 1986 , 1987 , 1989